# python

In [ ]:
import re
import shutil
from configparser import ConfigParser
from copy import deepcopy
from pathlib import Path

import importnb
import yaml
from packaging.requirements import Requirement
from packaging.utils import canonicalize_name

with importnb.Notebook():
    from jupyak.tasks import _actions as A
    from jupyak.tasks import _well_known as W
    from jupyak.tasks import _yak as Y

try:
    import tomllib
except ImportError:
    import tomli as tomllib

In [ ]:
def py_repo_tasks(repo: Y.Repo):
    yak = repo.parent
    py = repo.py
    work_path, in_repo = repo.run_context
    site_packages = yak.env.py_site_packages

    file_dep = [
        *A.resolve_globbish(work_path, py.file_dep),
        *[site_packages / _pth_path(dep) for dep in py.dependencies],
        yak.env.venv_history,
    ]

    for ppt in py.pyproject_tomls:
        yield from ppt_tasks(work_path / ppt, repo, file_dep)

    for ext_dir, info in py.lab_extensions.items():
        ext_path = work_path / ext_dir
        in_ext = deepcopy(in_repo)
        in_ext["cwd"] = ext_path
        script = py.labextension_script
        file_dep = A.resolve_globbish(work_path, info["file_dep"])
        file_dep += [script]
        targets = A.resolve_globbish(work_path, info["targets"])
        yield dict(
            name=f"{repo.name}:labext:{ext_path.name}",
            doc=f"> build the labextension {ext_path.name} from {repo.name}",
            actions=[A.run(["python", script, "build", "--debug", "."], in_ext)],
            file_dep=file_dep,
            targets=targets,
        )

> tasks for a single `pyproject.toml` (some repos have multiple) 

In [ ]:
def ppt_tasks(ppt: Path, repo: Y.Repo, file_dep: list[Path]):
    yak = repo.parent
    py = repo.py
    lite = repo.lite
    work_path, in_repo = repo.run_context
    in_dir = deepcopy(in_repo)
    in_dir.update(cwd=ppt.parent)
    name = ppt.parent.name
    site_packages = yak.env.py_site_packages
    pth = site_packages / _pth_path(name)
    globs = [f"__editable__.{name}-*.*"]
    pym = [*yak.env.run_args, "python3", "-m"]
    pip = [*pym, "pip"]
    e_args = ["-vv", "--no-deps", "--ignore-installed", "--no-build-isolation"]

    yield dict(
        name=f"{repo.name}:pip:{name}",
        doc=f"> do an editable python install for {name}",
        actions=[
            (A.clean, [pth], {"globs": {site_packages: globs}}),
            A.run([*pip, "install", "-e", ".", *e_args], in_dir),
            (_munge_pth, [name, repo]),
        ],
        file_dep=[*file_dep, ppt],
        targets=[pth],
    )

    dep_file = get_dep_file(ppt, yak)

    yield dict(
        name=f"{repo.name}:deps:{name}",
        doc=f"> extract the PyPI deps for {name}",
        actions=[(extract_py_deps, [ppt, dep_file])],
        file_dep=[ppt],
        targets=[dep_file],
    )

    if (
        lite
        and lite.wheel
        and not any(re.match(p, name) for p in lite.skip_wheel_patterns)
    ):
        dist = ppt.parent / "dist"
        shasums = dist / W.SHA256SUMS
        wheel_args = [
            *yak.env.run_args,
            "pyproject-build",
            ".",
            "--wheel",
            "--no-isolation",
        ]
        yield dict(
            name=f"{repo.name}:wheel:{name}",
            actions=[
                (A.clean, [dist]),
                A.run([*wheel_args], in_dir),
                (A.sha256_some, [shasums, dist, ["*.whl"]]),
            ],
            targets=[shasums],
            file_dep=[
                ppt,
                yak.env.venv_history,
                *A.resolve_globbish(work_path, lite.wheel_file_dep.get(name, [])),
            ],
        )

In [ ]:
def get_dep_file(pyproject_toml: Path, yak: Y.Yak):
    return yak.work_path / W.JPYK_PIP_DEPS / f"{pyproject_toml.parent.name}.yml"

In [ ]:
def _pth_path(name: str):
    return f"""_{name.replace("-", "_")}.pth"""

In [ ]:
def _munge_pth(name: str, repo: Y.Repo):
    print(f"   ... munging .pth for {name}")
    yak = repo.parent
    sp_dir = yak.env.py_site_packages
    pth_file = sp_dir / _pth_path(name)
    weird_pth = sorted(sp_dir.glob(f"__editable__.{name}-*.pth"))
    weird_egg = sorted(sp_dir.glob(f"{name.replace('_', '-')}.egg-link"))
    if weird_pth:
        print("      ... replacing", weird_pth[0].name, "with", pth_file.name)
        if pth_file.exists():
            pth_file.unlink()
        shutil.move(weird_pth[0], pth_file)
        weird_finder = sorted(sp_dir.glob(f"__editable___{name}_*_finder.py"))
    elif weird_egg:
        print("      ... replacing", weird_egg[0].name, "with", pth_file.name)
        if pth_file.exists():
            pth_file.unlink()
        shutil.move(weird_egg[0], pth_file)
    else:
        print(f"      ... no .egg-link or .pth to munge for {name}!")

try to get the required python runtime deps of a python project description.

In [ ]:
def extract_py_deps(pyproject_toml: Path, dep_file: Path):
    txt = tomllib.loads(pyproject_toml.read_text(encoding="utf-8"))
    raw_deps = []
    try:
        raw_deps += txt["build-system"]["requires"]
    except KeyError:
        print(f"   ... {pyproject_toml} has no `build-system`")
    try:
        raw_deps += txt["project"]["dependencies"]
    except KeyError:
        print(f"   ... {pyproject_toml} has no `dependencies`")

    setup_cfg = pyproject_toml.parent / "setup.cfg"
    if setup_cfg.exists():
        parser = ConfigParser()
        parser.read_string(setup_cfg.read_text(encoding="utf-8"))
        try:
            raw_deps += parser.get("options", "install_requires").strip().splitlines()
        except:
            print(
                "   ... {setup_cfg} exists, but contains no `options.install_requires`",
            )

    deps = {}
    for dep in sorted(raw_deps):
        req = Requirement(dep)
        if req.marker and not req.marker.evaluate():
            print(f"   ... {req} not needed for this platform")
            continue
        deps[canonicalize_name(req.name.strip())] = str(req.specifier)
    dep_file.parent.mkdir(parents=True, exist_ok=True)
    dep_file.write_text(yaml.safe_dump({"dependencies": deps}))